# Рекомендательная система на библиотеке surprise

Сайт библиотеки: http://surpriselib.com/

Установки библиотеки:
```
$ pip install scikit-surprise```
или
```
$ conda install -c conda-forge scikit-surprise```

In [1]:
import pandas as pd
import numpy as np

In [3]:
conda install -c conda-forge scikit-surprise

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\HP\Anaconda3

  added / updated specs:
    - scikit-surprise


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.11.0               |   py37h03978a9_0        16.9 MB  conda-forge
    scikit-surprise-1.1.1      |   py37h3a3b6f7_1         564 KB  conda-forge
    ------------------------------------------------------------
                                           Total:        17.5 MB

The following NEW packages will be INSTALLED:

  scikit-surprise    conda-forge/win-64::scikit-surprise-1.1.1-py37h3a3b6f7_1

The following packages will be UPDATED:

  conda              pkgs/main::conda-4.10.3-py37haa95532_0 --> conda-forge::conda-4.11.0-py37h03978a9_0




scikit-surprise-1.1. | 564 KB    |            |   0% 
scikit-surprise-1.1. | 564 KB    | 2          |   3% 
scikit-surprise-1.1. | 564 K

In [4]:
df = pd.read_csv("recdemo.csv", sep=";") #небольшой dataset

In [5]:
df

,id,A,B,C,D,E,F
0,1,3.0,4.0,2.0,4.0,1.0,NaN
1,2,3.0,4.0,2.0,4.0,NaN,2.0
2,3,NaN,2.0,5.0,5.0,NaN,5.0
3,4,NaN,NaN,NaN,NaN,4.0,NaN
4,5,3.0,NaN,2.0,4.0,4.0,NaN
5,6,NaN,5.0,5.0,5.0,NaN,5.0
6,7,1.0,NaN,NaN,2.0,NaN,3.0
7,8,NaN,NaN,NaN,NaN,4.0,4.0


In [6]:
df_unpivot=pd.melt(df, id_vars=['id'])
df_unpivot.head()

,id,variable,value
0,1,A,3.0
1,2,A,3.0
2,3,A,NaN
3,4,A,NaN
4,5,A,3.0


In [7]:
df_unpivot.dropna(inplace=True)
df_unpivot.columns=['userID', 'itemID', 'rating']

In [8]:
from surprise import Dataset
from surprise import Reader

reader = Reader(rating_scale=(1, 5)) # Зададим разброс оценок
data = Dataset.load_from_df(df_unpivot[['userID', 'itemID', 'rating']], reader) #создадим объект, с которым умеет работать библиотека

In [9]:
trainset = data.build_full_trainset()
testset = trainset.build_anti_testset()

In [10]:
testset[0:10]

[(1, 'F', 3.4642857142857144),
 (2, 'E', 3.4642857142857144),
 (5, 'B', 3.4642857142857144),
 (5, 'F', 3.4642857142857144),
 (7, 'B', 3.4642857142857144),
 (7, 'C', 3.4642857142857144),
 (7, 'E', 3.4642857142857144),
 (3, 'A', 3.4642857142857144),
 (3, 'E', 3.4642857142857144),
 (6, 'A', 3.4642857142857144)]

In [11]:
samples = [[0., 0., 0.], [0., .5, 0.], [1., 1., .5], [0.3, .5, 0.2], [.2, 1., .5]]
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(n_neighbors=2)
knn.fit(samples)
print(knn.kneighbors([[1., 1., 1.]]))

(array([[0.5       , 0.94339811]]), array([[2, 4]], dtype=int64))


In [12]:
print(knn.kneighbors([[1., 0., 1.]]))

(array([[1.11803399, 1.17473401]]), array([[2, 3]], dtype=int64))


In [13]:
from surprise import KNNBaseline
algo = KNNBaseline(k=1)
algo.fit(trainset)
predictions = algo.test(testset)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


In [14]:
predictions[0:5]

[Prediction(uid=1, iid='F', r_ui=3.4642857142857144, est=1.9577925900897966, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid=2, iid='E', r_ui=3.4642857142857144, est=1.0422074099102034, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid=5, iid='B', r_ui=3.4642857142857144, est=4.082661845573158, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid=5, iid='F', r_ui=3.4642857142857144, est=2.0826618455731576, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid=7, iid='B', r_ui=3.4642857142857144, est=3.8701641215302014, details={'actual_k': 1, 'was_impossible': False})]

In [15]:
df_unpivot1 = df_unpivot.copy()
for i in predictions:
    df_unpivot1 = df_unpivot1.append({'userID':i.uid, 'itemID': i.iid, 'rating': i.est}, ignore_index=True)

In [16]:
df_unpivot1.pivot(index='userID', columns='itemID', values='rating')

itemID,A,B,C,D,E,F
userID,,,,,,
1,3.000000,4.000000,2.000000,4.000000,1.000000,1.957793
2,3.000000,4.000000,2.000000,4.000000,1.042207,2.000000
3,3.311189,2.000000,5.000000,5.000000,4.090446,5.000000
4,3.072513,4.197382,2.072513,4.072513,4.000000,3.976639
5,3.000000,4.082662,2.000000,4.000000,4.000000,2.082662
6,3.469084,5.000000,5.000000,5.000000,4.248341,5.000000
7,1.000000,3.870164,1.870164,2.000000,3.691628,3.000000
8,3.095874,1.909554,2.095874,4.095874,4.000000,4.000000


In [17]:
algo = KNNBaseline(k=3)
algo.fit(trainset)
# Than predict ratings for all pairs (u, i) that are NOT in the training set.
predictions = algo.test(testset)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


In [18]:
df_unpivot3 = df_unpivot.copy()
for i in predictions:
    df_unpivot3 = df_unpivot3.append({'userID':i.uid, 'itemID': i.iid, 'rating': i.est}, ignore_index=True)
df_unpivot3.pivot(index='userID', columns='itemID', values='rating')

itemID,A,B,C,D,E,F
userID,,,,,,
1,3.000000,4.000000,2.000000,4.000000,1.000000,2.507435
2,3.000000,4.000000,2.000000,4.000000,2.601745,2.000000
3,3.256100,2.000000,5.000000,5.000000,3.527693,5.000000
4,3.083864,4.197382,2.083864,4.083864,4.000000,3.976639
5,3.000000,3.834977,2.000000,4.000000,4.000000,3.013312
6,3.418235,5.000000,5.000000,5.000000,3.590444,5.000000
7,1.000000,3.366011,1.857717,2.000000,3.095321,3.000000
8,2.579863,3.471928,3.463240,4.463240,4.000000,4.000000


## Кросс-валидация

In [19]:
from surprise.model_selection import cross_validate

In [20]:
cross_validate(algo, data, cv=2, verbose=True)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBaseline on 2 split(s).

                  Fold 1  Fold 2  Mean    Std     
RMSE (testset)    1.3616  1.3188  1.3402  0.0214  
MAE (testset)     1.0285  0.9399  0.9842  0.0443  
Fit time          0.01    0.00    0.00    0.00    
Test time         0.00    0.00    0.00    0.00    


{'test_rmse': array([1.36156113, 1.31876739]),
 'test_mae': array([1.02848654, 0.93989624]),
 'fit_time': (0.005998849868774414, 0.0020051002502441406),
 'test_time': (0.0009996891021728516, 0.0009963512420654297)}

In [21]:
cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


{'test_rmse': array([1.18141337, 1.64493972, 1.64685651]),
 'fit_time': (0.0020008087158203125,
  0.0010023117065429688,
  0.0011005401611328125),
 'test_time': (0.0010006427764892578, 0.0009999275207519531, 0.0)}

In [22]:
for i in range(1,6):
    algo = KNNBaseline(k=i, verbose=False) #отключим вывод логирования
    cv=cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)
    print(str(i)+'NN:',np.mean(cv['test_rmse']))

1NN: 1.5998308731538238
2NN: 1.1971293029337056
3NN: 1.6613526909761456
4NN: 1.4644119748009492
5NN: 1.54300931870935


## Item-based подход

In [23]:
algo = KNNBaseline(k=5, verbose=False)
predictions = algo.fit(trainset).test(testset)
df_unpivot5 = df_unpivot.copy()
for i in predictions:
    df_unpivot5 = df_unpivot5.append({'userID':i.uid, 'itemID': i.iid, 'rating': i.est}, ignore_index=True)
df_unpivot5.pivot(index='userID', columns='itemID', values='rating')

itemID,A,B,C,D,E,F
userID,,,,,,
1,3.000000,4.000000,2.000000,4.000000,1.000000,2.810335
2,3.000000,4.000000,2.000000,4.000000,2.601745,2.000000
3,2.859638,2.000000,5.000000,5.000000,3.527693,5.000000
4,3.083864,4.197382,2.083864,4.083864,4.000000,3.976639
5,3.000000,3.918340,2.000000,4.000000,4.000000,3.239815
6,3.053741,5.000000,5.000000,5.000000,3.590444,5.000000
7,1.000000,3.566434,2.632849,2.000000,3.095321,3.000000
8,2.615467,3.529529,3.297505,4.043101,4.000000,4.000000


In [24]:
sim_options = {'user_based': False } # Item-based подход

In [25]:
algo = KNNBaseline(k=5,sim_options=sim_options, verbose=False)
predictions = algo.fit(trainset).test(testset)
df_unpivot5_items = df_unpivot.copy()
for i in predictions:
    df_unpivot5_items = df_unpivot5_items.append({'userID':i.uid, 'itemID': i.iid, 'rating': i.est}, ignore_index=True)
df_unpivot5_items.pivot(index='userID', columns='itemID', values='rating')

itemID,A,B,C,D,E,F
userID,,,,,,
1,3.000000,4.000000,2.000000,4.000000,1.000000,2.329176
2,3.000000,4.000000,2.000000,4.000000,2.376574,2.000000
3,3.858465,2.000000,5.000000,5.000000,4.678967,5.000000
4,3.818825,4.122894,3.956402,4.261502,4.000000,4.155852
5,3.000000,3.337475,2.000000,4.000000,4.000000,3.311002
6,4.698465,5.000000,5.000000,5.000000,4.870350,5.000000
7,1.000000,1.789695,2.155844,2.000000,2.383400,3.000000
8,3.740899,4.021251,3.835183,4.150897,4.000000,4.000000


## Косинусная-мера

In [26]:
algo = KNNBaseline(k=5,sim_options= {'name': 'cosine'}, verbose=False)
predictions = algo.fit(trainset).test(testset)
df_unpivot5_cos = df_unpivot.copy()
for i in predictions:
    df_unpivot5_cos = df_unpivot5_cos.append({'userID':i.uid, 'itemID': i.iid, 'rating': i.est}, ignore_index=True)
df_unpivot5_cos.pivot(index='userID', columns='itemID', values='rating')

itemID,A,B,C,D,E,F
userID,,,,,,
1,3.000000,4.000000,2.000000,4.000000,1.000000,3.572362
2,3.000000,4.000000,2.000000,4.000000,2.927003,2.000000
3,2.754335,2.000000,5.000000,5.000000,3.270544,5.000000
4,3.134947,4.197382,2.134947,4.134947,4.000000,3.976639
5,3.000000,3.668772,2.000000,4.000000,4.000000,3.714426
6,2.939492,5.000000,5.000000,5.000000,3.360864,5.000000
7,1.000000,3.442211,2.951617,2.000000,2.802029,3.000000
8,2.700881,3.765123,3.231273,4.431273,4.000000,4.000000


## Datalens

В библиотеке есть встроенные Dataset. Можно воспользоваться ими.

Возьмем Dataset movielens (Побробнее о нем https://grouplens.org/datasets/movielens/ )

In [28]:
data = Dataset.load_builtin('ml-100k')#загружем данные о фильмах

Dataset ml-100k could not be found. Do you want to download it? [Y/n] ml-100k
Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\HP/.surprise_data/ml-100k


In [29]:
df = pd.DataFrame(data.raw_ratings)
df.columns = ['user', 'item', 'rating', 'timestamp']
df.head()

,user,item,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [30]:
df['user'].nunique(), df['item'].nunique()

(943, 1682)

In [31]:
for i in [1,3,5,7,20]:
    algo = KNNBaseline(k=i, verbose=False) #отключим вывод логирования
    cv=cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)
    print(str(i)+'NN:',np.mean(cv['test_rmse']))

1NN: 1.2463978132789109
3NN: 1.0387182482330322
5NN: 0.9945516116008767
7NN: 0.9732727541517154
20NN: 0.9401535890019249


## Задание:
Все задание выполняется на датасэтет 'ml-100k' (Dataset.load_builtin('ml-100k'))
1. Найдте для фильма алгоритмом kNN(k=20) с id 181 все неизвестные оценки (user-based). В ответе: посчитайте среднее полученных оценок.
2. Для параметров k = [10,20,30,40] найдите лучший item-based алгоритм (по метрике RMSE, кросс-валидация на 3 фолда).
Ответ: укажите k
3. Для лучшего алгоритма из пункта 2 найдте для фильма с id 181 все неизвестные оценки. В ответе: посчитайте среднее полученных оценок.

## Задача 1

In [32]:
data = Dataset.load_builtin('ml-100k')
trainset = data.build_full_trainset()
testset = trainset.build_anti_testset()

In [33]:
algo = KNNBaseline(k=20, verbose=False)
predictions = algo.fit(trainset).test(testset)

In [34]:
df_taks=pd.DataFrame(columns=['userID', 'itemID', 'rating'])

In [38]:
for i in predictions:
    if int(i.iid)==181:
        df_taks = df_taks.append({'userID':i.iid, 'itemID':i.iid, 'rating':i.est}, ignore_index=True)

In [39]:
df_taks['rating'].mean()

3.952208342677188

## Задача 2

In [40]:
argmin=5
k=0
for i in [10,20,30,40]:
    algo = KNNBaseline(k=i,sim_options= {'user_base': False}, verbose=False)
    cv=cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)
    if argmin>np.mean(cv['test_rmse']):
        k=i
        argmin=np.mean(cv['test_rmse'])
    print(str(i)+'NN', np.mean(cv['test_rmse']))

10NN 0.954950413483901
20NN 0.9411708573843386
30NN 0.9379065666680236
40NN 0.9351019311795711


In [41]:
print(k)

40


## Задача 3

In [42]:
algo = KNNBaseline(k=k, sim_options = {'user_base': False}, verbose=False)
predictions = algo.fit(trainset).test(testset)

In [43]:
df_taks=pd.DataFrame(columns=['userID', 'itemID', 'rating'])
for i in predictions:
    if int(i.iid)==181:
        df_taks = df_taks.append({'userID':i.iid, 'itemID':i.iid, 'rating':i.est}, ignore_index=True)

In [45]:
df_taks['rating'].mean()

nan